# Galda tenisa reitings

GT_1.5
    
    * Nav iespējas papildināt jau esošu datubāzi
    * Nav pievienoti PDF atrodamie dati (neregulāri ziņotie) sākot ar 2001. gadu (http://old.lgtf.lv/?id=57 #vīriešiem, http://old.lgtf.lv/?id=58 #sievietēm)
    * Nav pievienoti dati no http://www.galdateniss.lv/lat/latvijas-labakie/?offset=0


### Vēl dati vīriešiem:
http://old.lgtf.lv/?datums=0&datums=201312&serach_r=&id=27&said=0
http://old.lgtf.lv/?datums=201312&datums=201311&serach_r=&id=27&said=0
http://old.lgtf.lv/?datums=201311&datums=201310&serach_r=&id=27&said=0

### Vēl dati sievietēm:
http://old.lgtf.lv/?datums=0&datums=201312&serach_r=&id=28&said=0
http://old.lgtf.lv/?datums=201312&datums=201311&serach_r=&id=28&said=0
http://old.lgtf.lv/?datums=201311&datums=201310&serach_r=&id=28&said=0

In [1]:
#Vēlējos scrapot tieši mājaslapas tabulas
#no:
#https://lgtf.lv/#/viewpage/9 #Vīrieši
#https://lgtf.lv/#/viewpage/10 #Sievietes
#
#Papētot noskaidroju, ka iespējams iegūt JSON failu katrai redzamajai (uzspiežamajai) tabulai izmantojot POST

In [1]:
# Nepieciešamo bibliotēku imports
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
import numpy as np
import sqlite3 as sqlite
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Nav obligāti importēt.
from time import sleep
from time import strftime

### Datu scrapp ar post sagatavošana

In [2]:
#Adrese kur iegūstami ikmēneša dati no 2014...2022
my_url = "https://lgtf.lv/api/getRanking"

In [3]:
#Laika periods, gados (senāki dati tikai PDF)
each_year = range(2014, int(strftime('%Y')) + 1)

In [4]:
#Lai iegūtu datus no web
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)",
    "Accept": "text/html",
    "Accept-Charset": "utf-8",
    "Accept-Encoding": "none",
    "Accept-Language": "en-US",
    "Connection": "keep-alive"}

## Datu scrapp un SQLite tabulu papildināšana (pirmreizējā)

In [5]:
#Funkcija datu db sakārtošanai
def B_ranking_json_post(response):
    data = response.json()
    #Pilnā ielasītā DB
    df_full = pd.DataFrame.from_dict(data["players"])
       
    df_full.rename({"dzimsanas_dat": "birth_date", "licence_info": "license_till", "vieta": "place"}, axis = 1, inplace = True)
    df_full = df_full.astype({"player_id": object, "place": int, "points": int, "points_with_bonus": int})
    df_full.license_till.replace(to_replace = ["Nav", "Apmaksāta", "Neapmaksāta"], value = np.nan, inplace = True) 
    df_full["license_till"] = pd.to_datetime(df_full["license_till"], infer_datetime_format=True)
    df_full["birth_date"] = pd.to_datetime(df_full["birth_date"], errors="coerce",format="%Y-%m-%d")
    df_full["last_played_date"] = pd.to_datetime(df_full["last_played_date"], errors="coerce",format="%Y-%m-%d")
    #Šis, jo savādāk datos atkārtojas Player ID (atstāju augstvērtīgāko):
    df_full = df_full.groupby(["player_id"], as_index = False).agg(max)
    
    #DB spēlētāju info
    df_player = df_full[[
        "player_id",
        "name",
        "surname",
        "birth_date",
        "club_name",
        "license_till",
        "last_played_date"]].copy()
    
    #DB spēlētāja vieta pēc punkti + bonuss
    df_data = df_full[[
        "player_id",
        "place",
        "points_with_bonus",
        "points"]].copy()
    df_data.insert(4, "bonus_points", df_full.points_with_bonus - df_full.points)
    
    return df_player, df_data

# Pirmreizējie dati (ja datu nav)

In [6]:
#Cikls, kas sagatavo = scrappo datus => Pirmreizējais
C_db_player = pd.DataFrame()
C_db_data = pd.DataFrame()
C_date_list = [] #Pievienoju, sagatavoju datu grafiskai apstrādei
C_gender = ["virietis", "sieviete"]

for gender in C_gender:
    for years in each_year:  #Katrs pieejamais gads
        for months in range(1, 13):  #Katrs mēnesis gadā
            date_for_post = "-".join([str(years), str(months).zfill(2)])  #Sagatavojam str, lai tas būtu korekts post sadaļai
            if years == int(strftime('%Y')) and months == int(strftime('%m')) + 1:  #Izņēmums, ja sagatavotais "date_for_post" ir nākotnē
                break
            C_date_list.append(date_for_post)
            data = {
                "date": date_for_post, #2014-01 ... 2022-11"
                "gender": gender #"sieviete" / "virietis"
            }
            response = requests.post(my_url, headers=headers, data=data, timeout=2)
            if not response.ok:
                print(f"Kļūda datu ielasīšanā no {date_for_post}")
                continue
        
            C_db_player_each, C_db_data_each = B_ranking_json_post(response) #izgūstam apskatāmos df
        
            C_db_player_each.insert(1, "Gender", gender)
        
            if C_db_player.empty: 
                C_db_player = C_db_player_each #Ja 1. ieraksts
            else:
                C_db_player = pd.concat([C_db_player, C_db_player_each], axis=0, join="outer").drop_duplicates() #Pievienoju nākamo kolonnu
            
            C_db_data_each.insert(1,'Date', date_for_post)
            C_db_data_each = C_db_data_each.sort_values('points', ascending=False)
            C_db_data_each.insert(4,'place_by_true_points', range(1, 1 + len(C_db_data_each)))
            C_db_data_each = C_db_data_each.sort_values('bonus_points', ascending=False)
            C_db_data_each.insert(6,'place_by_bonus_points', range(1, 1 + len(C_db_data_each)))
            C_db_data_each.insert(4,'better_than', (1 - ((C_db_data_each.place - 1)/ (C_db_data_each.count() - 1)["player_id"])))
            C_db_data_each.insert(7,'better_than_by_TP', (1 - ((C_db_data_each.place_by_true_points - 1)/ (C_db_data_each.count() - 1)["player_id"])))
            C_db_data_each.insert(10,'better_than_by_BP', (1 - ((C_db_data_each.place_by_bonus_points - 1)/ (C_db_data_each.count() - 1)["player_id"])))
        
            if C_db_data.empty:
                C_db_data = C_db_data_each #Ja 1. ieraksts
            else:
                C_db_data = pd.concat([C_db_data, C_db_data_each], axis=0, join="outer") #Pievienoju nākamo
                
C_db_player.replace(to_replace="virietis", value="M", inplace=True)
C_db_player.replace(to_replace="sieviete", value="F", inplace=True)

# Datu papildināšanas funkcija (ja dati ir):

In [15]:
#Datu papildināšana:
#Cikls, kas sagatavo = scrappo datus => Pirmreizējais
C_db_player = pd.read_csv("from Jupiter to BI/all_db_player.csv" ,sep = ",")
C_db_player = C_db_player[C_db_player.columns.drop("Unnamed: 0")]
C_db_data = pd.read_csv("from Jupiter to BI/all_db_data.csv" ,sep = ",")
C_db_data = C_db_data[C_db_data.columns.drop("Unnamed: 0")]
C_date_list = pd.read_csv("from Jupiter to BI/all_db_data.csv" ,sep = ",", usecols = ["Date"])
C_date_list = C_date_list['Date'].values[-1].split('-') #Pievienoju, sagatavoju datu grafiskai apstrādei


C_gender = ["virietis", "sieviete"]

for gender in C_gender:
    for years in each_year:  #Katrs pieejamais gads
        for months in range(1, 13):  #Katrs mēnesis gadā
            date_for_post = "-".join([str(years), str(months).zfill(2)])  #Sagatavojam str, lai tas būtu korekts post sadaļai
            if years == int(strftime('%Y')) and months == int(strftime('%m')) + 1:  #Izņēmums, ja sagatavotais "date_for_post" ir nākotnē
                break
            if years < int(C_date_list[0]): 
                print('Šie dati jau ir')
                break
            if years == int(C_date_list[0]) and months <= int(C_date_list[1]):
                print('Šie dati jau ir')
                continue
            
            data = {
                "date": date_for_post, #2014-01 ... 2022-11"
                "gender": gender #"sieviete" / "virietis"
            }
            response = requests.post(my_url, headers=headers, data=data, timeout=2)
            if not response.ok:
                print(f"Kļūda datu ielasīšanā no {date_for_post}")
                continue
        
            C_db_player_each, C_db_data_each = B_ranking_json_post(response) #izgūstam apskatāmos df
        
            C_db_player_each.insert(1, "Gender", gender)
        
            C_db_player = pd.concat([C_db_player, C_db_player_each], axis=0, join="outer").drop_duplicates(subset = 'player_id') #Pievienoju nākamo kolonnu
            
            C_db_data_each.insert(1,'Date', date_for_post)
            C_db_data_each = C_db_data_each.sort_values('points', ascending=False)
            C_db_data_each.insert(4,'place_by_true_points', range(1, 1 + len(C_db_data_each)))
            C_db_data_each = C_db_data_each.sort_values('bonus_points', ascending=False)
            C_db_data_each.insert(6,'place_by_bonus_points', range(1, 1 + len(C_db_data_each)))
            C_db_data_each.insert(4,'better_than', (1 - ((C_db_data_each.place - 1)/ (C_db_data_each.count() - 1)["player_id"])))
            C_db_data_each.insert(7,'better_than_by_TP', (1 - ((C_db_data_each.place_by_true_points - 1)/ (C_db_data_each.count() - 1)["player_id"])))
            C_db_data_each.insert(10,'better_than_by_BP', (1 - ((C_db_data_each.place_by_bonus_points - 1)/ (C_db_data_each.count() - 1)["player_id"])))
        
            C_db_data = pd.concat([C_db_data, C_db_data_each], axis=0, join="outer") #Pievienoju nākamo
                
C_db_player.replace(to_replace="virietis", value="M", inplace=True)
C_db_player.replace(to_replace="sieviete", value="F", inplace=True)

Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir
Šie dati jau ir


In [16]:
C_db_player

,player_id,Gender,name,surname,birth_date,club_name,license_till,last_played_date
0,27,M,Valērijs,Samoilovs,1938-12-30,GTK Namejs,2023-03-19,2022-11-20
1,28,M,Viktors,Krūmiņš,1943-08-17,Latvijas US,NaN,2022-11-12
2,29,M,Aivars,Kamols,1935-08-12,Bauskas novads,2023-10-15,2022-10-22
3,30,M,Grigorijs,Muntikovs,1942-10-23,Latvijas US,NaN,2020-08-29
4,31,M,Juris,Kravals,1954-09-01,Latvijas US,2023-07-16,2022-11-05
...,...,...,...,...,...,...,...,...
706,3924,M,Staņislavs,Ļitvaks,1977-01-26 00:00:00,GTK Namejs,NaT,2022-11-05 00:00:00
707,3925,M,Mikhail,Teslya,1988-01-03 00:00:00,GTK Namejs,2023-11-19 00:00:00,2022-12-18 00:00:00
708,3926,M,Andrejs,Gindins,1970-01-25 00:00:00,GTK Namejs,2023-11-26 00:00:00,2022-11-26 00:00:00
709,3931,M,Vjačeslavs,Lebedevs,1979-11-23 00:00:00,Bez kluba,NaT,2022-11-26 00:00:00


In [17]:
C_db_data

,player_id,Date,place,points_with_bonus,better_than,place_by_true_points,points,better_than_by_TP,place_by_bonus_points,bonus_points,better_than_by_BP
0,37,2014-01,14,1549,0.981612,27,1284,0.963225,1,265,1.000000
1,45,2014-01,33,1220,0.954738,45,990,0.937765,2,230,0.998586
2,49,2014-01,18,1532,0.975955,24,1304,0.967468,3,228,0.997171
3,216,2014-01,22,1440,0.970297,31,1225,0.957567,4,215,0.995757
4,176,2014-01,13,1562,0.983027,18,1372,0.975955,5,190,0.994342
...,...,...,...,...,...,...,...,...,...,...,...
88,1949,2022-12,94,14,0.363014,94,14,0.363014,143,0,0.027397
95,2000,2022-12,93,15,0.369863,93,15,0.369863,144,0,0.020548
94,1984,2022-12,92,16,0.376712,91,16,0.383562,145,0,0.013699
71,1670,2022-12,91,18,0.383562,89,18,0.397260,146,0,0.006849


### Saglabāšu CSV

In [18]:
C_db_player.to_csv("from Jupiter to BI/all_db_player.csv" ,sep = ",")
C_db_data.to_csv("from Jupiter to BI/all_db_data.csv" ,sep = ",")

In [19]:
with pd.ExcelWriter("from Jupiter to BI/db_data.xlsx") as writer:
    C_db_player.to_excel(writer, sheet_name="Players")  
    C_db_data.to_excel(writer, sheet_name="Data")